In [2]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_ps.csv', sep = ',')

In [3]:
melb_df['Address'].nunique()

13378

In [5]:
## Извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар)
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
    # Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
    # Метод split() разбивает строку на слова по пробелу.
    # В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
    # Обрезаем список, оставляя в нём только последний элемент,
    # потенциальный подтип улицы, и заносим в переменную street_type. 
    street_type = address_list[-1]
    # Делаем проверку на то, что полученный подтип является географической пометкой.
    # Для этого проверяем его на наличие в списке exclude_list. 
    if street_type in exclude_list:
    # Если переменная street_type является географической пометкой,
    # переопределяем её на второй элемент с конца списка address_list.
        return address_list[-2]
    return street_type

In [76]:
street_types = melb_df['Address'].apply(get_street_type)
melb_df['StreetTypes'] = street_types
melb_df['StreetTypes'].nunique()

56

In [77]:
def eliminate_longs(address):
    small_list = []
    for a in melb_df['StreetTypes'].unique:
        if len(a) <= 3:
            small_list.append(a)
    for b in small_list:
        if set(b).issubset(set(address)):
            return b
    return address

In [24]:
display(street_types.nunique())
street_types.value_counts()

St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Corso     

In [15]:
popular_stypes = street_types.value_counts().nlargest(10).index
display(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

In [21]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
melb_df['StreetType'].nunique()

11

In [23]:
melb_df.drop('Address', axis=1, inplace = True)